# Utility Appendix

This is the Utility File with all of the functions put in so that all the Juypter Notebooks can use any of these functions

In [1]:
import pandas as pd
import numpy as np
import cmath
import math
from scipy.optimize import fmin, minimize
from astropy import units as u
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pyplot as plt
import copy
%matplotlib inline

In [2]:
class data:
    u: float
    v: float
    phase: float
    amp: float
    sigma: float
    vis_data: complex
    def __init__(self, u, v, phase, amp, sigma):
        self.u = u
        self.v = v
        self.phase = phase
        self.amp = amp
        self.sigma = sigma
        self.vis_data = amp * np.exp(1j * math.radians(phase))

    def __repr__(self):
        return f"[u: {self.u}, v: {self.v}]"

    def __str__(self):
        return f"[u: {self.u}, v: {self.v}]"

In [3]:
def process_data(data_df):
    """
    Processes the data in the dataframe into a coords list and data objects
    Args:
        data_df is a pandas data frame of the data
    Returns:
        a list of coordinates in u,v space
        a list of data objects
    """

    coords = []
    data_list = []
    for i in range(len(data_df)):
        data_list.append(data(data_df.loc[i, 'U(lambda)'], data_df.loc[i, 'V(lambda)'], data_df.loc[i, 'Iphase(d)'], data_df.loc[i, 'Iamp(Jy)'], data_df.loc[i, 'Isigma(Jy)']))
        coords.append([data_df.loc[i, 'U(lambda)'], data_df.loc[i, 'V(lambda)']])
    coords = np.array(coords)
    return coords, data_list

In [4]:
def read_data(filename):
    """
    reads the data from a file into a pandas dataframe
    Args:
        filename is a string that represents a csv file
    Returns:
        a pandas dataframe
    """
    df = pd.read_csv(filename)
    return df

In [5]:
df = read_data("./data/SR1_M87_2017_095_hi_hops_netcal_StokesI.csv")
coords, data_list = process_data(df)

In [6]:
def loss(image, data_list: list[data], coords, p = 2, reg_weight = 1, FOV = 100*u.uas.to(u.rad)):
    """
    calculates the loss of an image compared to the data given
    Args:
        image is a 80x80 pixel image that represents our reconstructed image
        coords is a list of u,v coordinates that we obtained from our data
        p is the kind of norm to be used
        reg_weight is the regularizer weight
        FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.
    Returns:
        a loss value
    """
    error_sum = 0
    vis_images = interpolate(image, coords, FOV)
    
    for i in range(len(data_list)):
        vis_data = data_list[i].amp * np.exp(1j * math.radians(data_list[i].phase))
        vis_image = vis_images[i]
        error = (abs(vis_image-vis_data) / data_list[i].sigma) ** 2
        error_sum += error
    
    return error_sum + reg_weight * calc_regularizer(image=image, tsv=True, p=2)

In [7]:
def do_sample(n):
    """
    Collects n samples from a sample image
    Args:
        n is the number of samples as an integer
    Returns:
        a list of coordinates in u,v space
        a list of data objects
    """
    coords = []
    data_list = []
    ft_image = np.fft.fftshift(np.fft.fft2(sample))
    for i in range(n):
        coords.append((int(np.random.rand()*10-5), int(np.random.rand()*10-5)))
        data_list.append(data(coords[i][0], coords[i][1], 0, ft_image[coords[i][0]+40][coords[i][1]+40], 1))
    return coords, data_list

In [8]:
def calculate_losses():
    """
    Calculates the losses of the image by shifting it around 
    Args:
        None
    Returns:
        an array of losses where the index is how much the image is shifted starting with -40 to 40
    """
    loss_arr = np.zeros((len(sample),len(sample[0])))
    for i in range(len(sample)):
        image_1 = np.roll(sample, i, axis=1) # Right shifts
        for j in range(len(sample[i])):
            image_2 = np.roll(image_1, j, axis = 0) # Up shifts
            loss_arr[i][j] = loss(image_2, data_list, coords, reg_weight=0, FOV=1)
            
    loss_arr1 = np.roll(loss_arr, 40, axis=1)
    loss_arr2 = np.roll(loss_arr1, 40, axis=0)

    plt.figure()
    plt.imshow(loss_arr2)
    plt.axis('off')
    return loss_arr

In [9]:
def interpolate(image, coords, FOV):
    """
    Interpolates the values of each coordinate in coords in the fourier domain of image
    Args:
        image is a 80x80 pixel image that represents our reconstructed image
        coords is a list of u,v coordinates that we obtained from our data
        FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.
    Returns:
        The interpolated values at the coordinates based on image
    """

    ft_image = np.fft.fftshift(np.fft.fft2(image))

    k_FOV = 1/FOV

    kx = np.fft.fftshift(np.fft.fftfreq(ft_image.shape[0], d = 1/(k_FOV*ft_image.shape[0])))
    ky = np.fft.fftshift(np.fft.fftfreq(ft_image.shape[1], d = 1/(k_FOV*ft_image.shape[1])))

    interp_real = RegularGridInterpolator((kx, ky), ft_image.real, bounds_error=False, method="linear")
    interp_imag = RegularGridInterpolator((kx, ky), ft_image.imag, bounds_error=False, method="linear")

    real = interp_real(coords)
    imag = interp_imag(coords)

    return real + imag * 1j

In [10]:
def calc_regularizer(image: np.array, tsv=False, p=None):
    """
    Calculates the regularizer according to total squared variation
    Args:
        image is a 80x80 pixel image that represents our reconstructed image
        p is the kind of norm to be used
        tsv is the flag for total squared variation
    Returns:
        the regularizer
    """
    if tsv and p == None:
        raise Exception("p value not set")
    reg = 0
    if tsv:
        image_lshift = np.copy(image, subok=True)
        image_lshift = np.roll(image_lshift, -1,axis=1)
        image_lshift[:,-1] = image_lshift[:,-2] 
        image_upshift = np.copy(image, subok=True)
        image_upshift = np.roll(image_upshift, -1, axis=0)
        image_upshift[-1] = image_upshift[-2] 

        term_1 = np.power(np.absolute(np.subtract(image_lshift, image)),p)
        term_2 = np.power(np.absolute(np.subtract(image_upshift, image)),p)
        reg = np.sum(np.add(term_1,term_2))
    return -1 * reg

In [11]:
def gradient_regularizer(image: np.array):
    """
    Calculates the gradient of the regularizer
    Args:
        image is a 80x80 pixel image that represents our reconstructed image
    Returns:
        the gradient of the regularizer
    """
    image_lshift = np.copy(image, subok=True)
    image_lshift = np.roll(image_lshift, -1,axis=1)
    image_lshift[:,-1] = image_lshift[:,-2] 
    image_upshift = np.copy(image, subok=True)
    image_upshift = np.roll(image_upshift, -1, axis=0)
    image_upshift[-1] = image_upshift[-2] 
    image_rshift = np.copy(image, subok=True)
    image_rshift = np.roll(image_rshift, 1,axis=1)
    image_rshift[:,0] = image_rshift[:,1] 
    image_dshift = np.copy(image, subok=True)
    image_dshift = np.roll(image_dshift, 1, axis=0)
    image_dshift[0] = image_lshift[1] 
    g_reg = 4 * image - image_lshift - image_upshift - image_rshift - image_dshift
    return g_reg

In [12]:
def gradient_finite_differences(data_list: list[data], coords, image, mode = 1, FOV = 100*u.uas.to(u.rad)):
    """
    Calculates a gradient based on finite differences
    Args:
        data_list is a list of data objects
        coords is a list of u,v coordinates that we obtained from our data
        image is a 80x80 pixel image that represents our reconstructed image
        mode is the type of difference used: 0 For central, -1 for backward, 1 for forward
        FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.
    Returns:
        the gradient of the loss function
    """
    image_copy = np.copy(image, subok=True)
    upper_diff: float
    lower_diff: float
    h: float
    gradient_arr = np.empty(np.shape(image),dtype=np.complex_)
    if (mode == 0): # Central difference
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-6 / 2
                upper_diff = loss(image_copy, data_list, coords, FOV=FOV)
                image_copy[row,col] -= 1e-6
                lower_diff = loss(image_copy, data_list, coords, FOV=FOV)
                image_copy[row,col] = image[row,col] # Reset that pixel to original value
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-6
    elif (mode == -1): # Backward difference
        upper_diff = loss(image, data_list, coords, FOV=FOV)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] -= 1e-8
                lower_diff = loss(image_copy, data_list, coords, FOV=FOV)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    elif (mode == 1) : # Forward difference is default
        lower_diff = loss(image, data_list, coords, FOV=FOV)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-8 
                upper_diff = loss(image_copy, data_list, coords, FOV=FOV)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    else:
        raise ValueError('Incorrect mode for finite differences')
    return gradient_arr.real

In [13]:
def gradient_descent(image, data_list, coords, coeffs = None, FOV = 100*u.uas.to(u.rad), stopper = None, dirty = False):
    """
    Performs gradient descent to reconstruct the image
    Args:
        image is a 80x80 pixel image that represents our reconstructed image
        data_list is a list of data objects
        coords is a list of u,v coordinates that we obtained from our data
        coeffs is the precomputed coefficients
        FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.
        stopper allows the descent to stop at 20 iterations
        dirty changes the gradient mode to dirty kernel
    Returns:
        the reconstructed image
    """
    image_copy = np.copy(image, subok=True) # Uses copy of the image due to lists being mutable in python
    i = 0
    grad = None
    # Can also use max here, min just makes it finish quicker
    while grad is None or np.min(np.abs(grad)) > 0.00001:
        t = 10000000 # Initial Step size which resets each iteration
        prev_loss = loss(image_copy, data_list, coords, FOV=FOV)

        if dirty:
            grad = dirty_gradient(data_list, coords, coeffs, image_copy)
        else:
            grad = gradient_finite_differences(data_list, coords, image_copy, FOV=FOV)

        new_image = image_copy - t * grad.real
        new_loss = loss(new_image, data_list, coords, FOV=FOV)
        
        while new_loss > prev_loss: # Only run when new_loss > prev_loss
            new_image = image_copy - t * grad.real
            new_loss = loss(new_image, data_list, coords, FOV=FOV)
            t /= 2

        image_copy -= t * 2 * grad.real # Multiply by 2 to undo last divide in the while loop
        i += 1
        if stopper != None:
            if i == stopper: # Hard stop here for notebook purposes
                return image_copy
        print("loss:",new_loss)
    return image_copy

In [14]:
def preprocess_gradient(data_list, coords, image):
    """
    Precomputed the coefficients decribed in dirty gradient
    Args:
        data_list is a list of data objects
        coords is a list of u,v coordinates that we obtained from our data
        image is a 80x80 pixel image that represents our reconstructed image
    Returns:
        a 4d list of coefficients
    """
    r, c = np.shape(image)
    preprocessed = np.empty([r,c,len(data_list),2], dtype=np.complex_)
    for row in range(len(image)):
        for col in range(len(image[row])):
            for datum in range(len(data_list)):
                term = ((2*np.pi*1j)/image.size)*(row*coords[datum][0] + col*coords[datum][1]) #.size for numpy array returns # of rows * # of cols
                term_1 = np.exp(term)
                term_2 = np.exp(-1*term)
                preprocessed[row,col,datum,0] = term_1
                preprocessed[row,col,datum,1] = term_2
    return preprocessed

In [15]:
def dirty_gradient(data_list: list[data], coords, coeffs, image, FOV = 100*u.uas.to(u.rad)):
    """
    Calculates a gradient based on dirting the image
    Args:
        data_list is a list of data objects
        coords is a list of u,v coordinates that we obtained from our data
        coeffs is the precomputed coefficients
        image is a 80x80 pixel image that represents our reconstructed image
        FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.
    Returns:
        the gradient of the loss function
    """
    gradient_arr = np.empty(np.shape(image)) # Because we are in real space
    vis_images = interpolate(image, coords, FOV)
    for row in range(len(image)):
        for col in range(len(image[row])):
            gradient_sum = 0
            for i in range(len(data_list)):
                vis_data = data_list[i].vis_data
                vis_image = vis_images[i] # Ask about this on Wednesday
                term_1 = coeffs[row,col,i,0] * (np.conj(vis_image) - np.conj(vis_data))
                term_2 = coeffs[row,col,i,1] * (vis_image - vis_data)
                gradient_sum += (term_1 + term_2)/(data_list[i].sigma ** 2)
            gradient_arr[row,col] = gradient_sum.real
    return gradient_arr